In [1]:
!gcloud config get-value project

fast-learner-project


In [2]:
!gcloud config set project fast-learner-project

Updated property [core/project].


In [3]:
!gcloud container clusters get-credentials two-node-cluster --zone us-central1-a

Fetching cluster endpoint and auth data.
kubeconfig entry generated for two-node-cluster.


In [63]:
# get kubectl nodes
nodes = !kubectl get nodes

In [64]:
'''
['NAME                                             STATUS   ROLES    AGE     VERSION',
 'gke-two-node-cluster-node-pool-1-6f09704a-0m73   Ready    <none>   9m44s   v1.33.2-gke.1240000',
 'gke-two-node-cluster-node-pool-1-6f09704a-8bcl   Ready    <none>   9m42s   v1.33.2-gke.1240000']
'''

# get pods
first_node = nodes[1].split()[0]
second_node = nodes[2].split()[0]

In [ ]:
# C:\Users\adilm\OneDrive\Documents\large-scale-online-learning\MLOps-Architecture\terraform_stuff\deploy_real_kubernetes\00-namespace.yaml
!kubectl apply -f deploy_real_kubernetes/00-namespace.yaml



namespace/kafka created


In [65]:
# change the node name of C:\Users\adilm\OneDrive\Documents\large-scale-online-learning\MLOps-Architecture\terraform_stuff\deploy_real_kubernetes\01-zookeeper.yaml
# using python logic

import yaml

dir_yaml = 'deploy_real_kubernetes/'

# FIRST NODE
with open(dir_yaml + 'deploy_micro_inf.yaml', 'r') as filename:
    micro_inf_configs = list(yaml.safe_load_all(filename))
micro_inf_configs[0]['spec']['template']['spec']['nodeName'] = first_node
with open(dir_yaml + 'deploy_micro_inf.yaml', 'w') as filename:
    yaml.safe_dump_all(micro_inf_configs, filename)

with open(dir_yaml + '04-mlflow.yaml', 'r') as filename:
    mlflow_configs = list(yaml.safe_load_all(filename))
mlflow_configs[2]['spec']['template']['spec']['nodeName'] = first_node
with open(dir_yaml + '04-mlflow.yaml', 'w') as filename:
    yaml.safe_dump_all(mlflow_configs, filename)

# SECOND NODE
with open(dir_yaml + '01-zookeeper.yaml', 'r') as filename:
    zookeeper_configs = list(yaml.safe_load_all(filename))
zookeeper_configs[1]['spec']['template']['spec']['nodeName'] = second_node
with open(dir_yaml + '01-zookeeper.yaml', 'w') as filename:
    yaml.safe_dump_all(zookeeper_configs, filename)

with open(dir_yaml + '02-kafka.yaml', 'r') as filename:
    kafka_configs = list(yaml.safe_load_all(filename))
kafka_configs[1]['spec']['template']['spec']['nodeName'] = second_node
with open(dir_yaml + '02-kafka.yaml', 'w') as filename: 
    yaml.safe_dump_all(kafka_configs, filename)

with open(dir_yaml + 'deploy_micro_up.yaml', 'r') as filename:
    micro_up_configs = list(yaml.safe_load_all(filename))
micro_up_configs[0]['spec']['template']['spec']['nodeName'] = second_node
with open(dir_yaml + 'deploy_micro_up.yaml', 'w') as filename:
    yaml.safe_dump_all(micro_up_configs, filename)



In [ ]:
# get all files from dir deploy_real_kubernetes
dir_name = 'deploy_real_kubernetes'
import os

files = [f for f in os.listdir(dir_name) if os.path.isfile(os.path.join(dir_name, f))]
for file in files:
    if file.endswith('.yaml'):
        print(f'Applying {file}...')
        !kubectl apply -f {os.path.join(dir_name, file)}
    else:
        print(f'Skipping {file}, not a YAML file.')

In [50]:
!kubectl apply -f deploy_real_kubernetes/deploy_micro_up.yaml

deployment.apps/api-update configured
persistentvolumeclaim/api-update-pvc unchanged


In [ ]:
!kubectl get pods 

In [ ]:
# open logs api-update-869db87fbd-mrmkv
!kubectl logs api-update-869db87fbd-mrmkv -n default

In [ ]:
# check kubectl event
!kubectl get events 

In [ ]:
!kubectl get storageclass

In [ ]:
!kubectl describe pvc api-update-pvc

In [54]:
!kubectl get pv

NAME                                       CAPACITY   ACCESS MODES   RECLAIM POLICY   STATUS   CLAIM                STORAGECLASS   VOLUMEATTRIBUTESCLASS   REASON   AGE
pvc-5760623a-2e56-4a7f-85ed-cced7d10f8ec   5Gi        RWO            Delete           Bound    default/mlflow-pvc   standard       <unset>                          43m


In [61]:
!kubectl get volumeattachment


NAME                                                                   ATTACHER                PV                                         NODE                                             ATTACHED   AGE
csi-1995fde2e78e9e9a3e250210be8f9432d0c786e44e0761a013f0ac72d9d3dba6   pd.csi.storage.gke.io   pvc-5760623a-2e56-4a7f-85ed-cced7d10f8ec   gke-two-node-cluster-node-pool-1-6f09704a-0m73   true       66m


In [62]:
!kubectl get nodes


NAME                                             STATUS   ROLES    AGE    VERSION
gke-two-node-cluster-node-pool-1-6f09704a-0m73   Ready    <none>   115m   v1.33.2-gke.1240000
gke-two-node-cluster-node-pool-1-6f09704a-8bcl   Ready    <none>   115m   v1.33.2-gke.1240000


In [ ]:
!kubectl get storageclass standard-rwo

In [59]:
!kubectl get pods -n kube-system 


NAME                                                        READY   STATUS    RESTARTS   AGE
event-exporter-gke-6c5c978549-msrmf                         2/2     Running   0          101m
fluentbit-gke-lc7dd                                         3/3     Running   0          95m
fluentbit-gke-r7fvs                                         3/3     Running   0          95m
gke-metrics-agent-5f5dr                                     2/2     Running   0          95m
gke-metrics-agent-dxdwf                                     2/2     Running   0          95m
konnectivity-agent-65cbfbd596-ln99r                         2/2     Running   0          94m
konnectivity-agent-65cbfbd596-r4mlt                         2/2     Running   0          101m
konnectivity-agent-autoscaler-77dfdd8f5b-wpvp9              1/1     Running   0          101m
kube-dns-85896794dd-4vk6t                                   4/4     Running   0          101m
kube-dns-85896794dd-hcfnd                                   4/4   

In [57]:
!kubectl get csidrivers


NAME                    ATTACHREQUIRED   PODINFOONMOUNT   STORAGECAPACITY   TOKENREQUESTS   REQUIRESREPUBLISH   MODES        AGE
pd.csi.storage.gke.io   true             false            false             <unset>         false               Persistent   101m


In [35]:
!kubectl describe pvc mlflow-pvc -n default


Name:          mlflow-pvc
Namespace:     default
StorageClass:  standard
Status:        Bound
Volume:        pvc-5760623a-2e56-4a7f-85ed-cced7d10f8ec
Labels:        <none>
Annotations:   pv.kubernetes.io/bind-completed: yes
               pv.kubernetes.io/bound-by-controller: yes
               volume.beta.kubernetes.io/storage-provisioner: pd.csi.storage.gke.io
               volume.kubernetes.io/storage-provisioner: pd.csi.storage.gke.io
Finalizers:    [kubernetes.io/pvc-protection]
Capacity:      5Gi
Access Modes:  RWO
VolumeMode:    Filesystem
Used By:       mlflow-6cbdd998d8-dbxjr
Events:
  Type    Reason                 Age                From                                                                                              Message
  ----    ------                 ----               ----                                                                                              -------
  Normal  ExternalProvisioning   23m (x2 over 23m)  persistentvolume-controller    